# Silicon calibration for a single crystal on ID11 nscope

Uses the older ImageD11 calibration fitting routines

Last updated 22/02/2025 by @jadball

In [ ]:
exec(open('/data/id11/nanoscope/install_ImageD11_from_git.py').read())
PYTHONPATH = setup_ImageD11_from_git( ) # os.path.join( os.environ['HOME'],'Code'), 'ImageD11_git' )

In [ ]:
# Experts : update these files for your detector if you need to
maskfile = "/data/id11/nanoscope/Eiger/eiger_mask_E-08-0144_20240205.edf"
e2dxfile = "/data/id11/nanoscope/Eiger/e2dx_E-08-0144_20240205.edf"
e2dyfile = "/data/id11/nanoscope/Eiger/e2dy_E-08-0144_20240205.edf"
detector = 'eiger'
omegamotor = 'rot_center'
dtymotor = 'dty'

# Default segmentation options
options = { 'cut' : 1, 'pixels_in_spot' : 3, 'howmany' : 100000 }

# EXPERTS: These can be provided as papermill parameters. Users, leave these as None for now...
dataroot = None
analysisroot = None
sample = None
dataset = None

econst = 12.398423  # energy conversion - don't change

In [ ]:
# 'Ag': 25.514, 'Sn': 29.2001, 'Nd': 43.5689, 'Gd': 50.2391, 'Hf': 65.3508, 'W' : 69.525, 'Pt': 78.3948, 'Pb': 88.0045
# here enter the lattice parameters and spacegroup of the calibrant
# at ID11 we use NIST SRM SiO2 670c

symmetry = "cubic"
a_silicon = 5.43094
energy = 43.5689  # your guess of the beam energy in keV

In [ ]:
%matplotlib ipympl
import pylab as pl
import numpy as np
import ImageD11.sinograms.dataset
import ImageD11.sinograms.assemble_label
import ImageD11.sinograms.properties
import ImageD11.transformer
import ImageD11.columnfile
import ImageD11.unitcell
import ImageD11.indexing
import ImageD11.grain
import ImageD11.nbGui.fit_geometry
import ImageD11.nbGui.nb_utils as utils
import silx.io
from ImageD11.parameters import AnalysisSchema, parameters
from ImageD11.nbGui import segmenter_gui

# Segment the single crystal data

In [ ]:
# Set up the file paths. Edit this if you are not at ESRF or not using the latest data policy.
if dataroot is None:
    dataroot, analysisroot = segmenter_gui.guess_ESRF_paths() 

if len(dataroot)==0:
    print("Please fix in the dataroot and analysisroot folder names above!!")
    
print('dataroot =',repr(dataroot))
print('analysisroot =',repr(analysisroot))

In [ ]:
# List the samples available:
segmenter_gui.printsamples(dataroot)

In [ ]:
# USER: Decide which sample
if sample is None:
    sample = 'Si_cube'

In [ ]:
# List the datasets for that sample:
segmenter_gui.printdatasets( dataroot, sample )

In [ ]:
# USER: Decide which dataset
if dataset is None:
    dataset = "rot"

In [ ]:
# create ImageD11 dataset object
dset = ImageD11.sinograms.dataset.DataSet(dataroot=dataroot,
                                        analysisroot=analysisroot,
                                        sample=sample,
                                        dset=dataset,
                                        detector=detector,
                                        omegamotor=omegamotor,
                                        dtymotor=dtymotor
                                       )
dset.import_all()  # Can use scans = [f'{scan}.1' for scan in range(1,102)] )
dset.maskfile = maskfile
dset.e2dxfile = e2dxfile
dset.e2dyfile = e2dyfile
dset.save()

In [ ]:
ui = segmenter_gui.SegmenterGui(dset, **options )

In [ ]:
options = ui.getopts()

In [ ]:
# create batch file to send to SLURM cluster
sbat = ImageD11.sinograms.lima_segmenter.setup(dset.dsfile, **ui.getopts(), pythonpath=PYTHONPATH)
if sbat is None:
    raise ValueError("This scan has already been segmented!")
print(sbat)

In [ ]:
utils.slurm_submit_and_wait(sbat, 60)

In [ ]:
# label sparse peaks

ImageD11.sinograms.assemble_label.main(dset.dsfile)

In [ ]:
# generate peaks table

ImageD11.sinograms.properties.main(dset.dsfile, options={'algorithm': 'lmlabel', 'wtmax': 70000, 'save_overlaps': False})

# View peaks

In [ ]:
# Load some peaks for your silicon

colf = dset.get_cf_2d()

In [ ]:
# Remove any weak peaks / noise (average intensity > cutoff)
cutoff = 10
colf.filter(colf.sum_intensity / colf.Number_of_pixels > cutoff)

In [ ]:
f, a = pl.subplots(1,2,figsize=(12,6), constrained_layout=True)
a[0].plot(colf.fc,colf.sum_intensity/colf.Number_of_pixels,'.',alpha=0.2)
a[0].set(yscale='log',xlabel='pixel',ylabel='intensity')
a[1].plot(colf.fc,colf.sc,'.')
a[1].set(xlabel='pixel', ylabel='pixel');
pl.show()

In [ ]:
# set up our filenames

flt_file = "si.flt"
gve_file = "si.gve"
new_flt_file = "si.flt.new"
par_file_start = "si_start.par"  # you can set this to an existing parameter file if you already ran the powder_calib.ipynb notebook
par_file_powder = "si_powder.par"
par_file_end = "si_fit.par"
ubi_file = "si.ubi"
map_file = "si.map"

In [ ]:
colf.writefile(flt_file)

In [ ]:
def auto_guess_distance(masterfile, scan):
    """
    Automatically guess the distance from the masterfile
    """
    possible_distance_motors = ['ffdtx1', 'frelx']
    distance_um = None
    for mot in possible_distance_motors:
        try:
            distance_um = float(silx.io.get_data(f"silx:{masterfile}::{scan}/instrument/positioners/{mot}" )) * 1e3  # microns
        except ValueError:
            continue
    if distance_um is None:
        raise ValueError("Couldn't find distance!")
    
    return distance_um

In [ ]:
# guess the detector distance in um
# you can also manually specify
distance_guess = auto_guess_distance(dset.masterfile, dset.scans[-1])  # detector distance in um
print(distance_guess)
# distance_guess = 140000  # 140 mm

__Note: If you followed powder_calib.ipynb first, you should have a much better starting point than the below suggested parameter file.__

__Currently the below cell is only set up for the Eiger__

In [ ]:
if not os.path.exists(par_file_start):
    with open(par_file_start,"w") as pars:
        pars.write(f"""cell__a {a_silicon}
cell__b {a_silicon}
cell__c {a_silicon}
cell_alpha 90.0
cell_beta 90.0
cell_gamma 90.0
cell_lattice_[P,A,B,C,I,F,R] 227
chi 0.0
distance {distance_guess}
fit_tolerance 0.05
o11 -1
o12 0
o21 0
o22 -1
omegasign 1.0
t_x 0
t_y 0
t_z 0
tilt_x 0.0
tilt_y 0.0
tilt_z 0.0
wavelength {econst/energy}
wedge 0.0
y_center 1062.0
y_size 75.0
z_center 1126.0
z_size 75.0""")

# Initial fit of parameters with Si peaks

In [ ]:
ui = ImageD11.nbGui.fit_geometry.FitGeom()
ui.loadfiltered(flt_file)
ui.loadfileparameters(par_file_start) #, phase_name='Si')  # you must specify a phase name if you're using an existing json file
ui.fitGui()

In [ ]:
ui.saveparameters(par_file_powder)
ui.savegv(gve_file)

# Indexing single crystal

In [ ]:
# Change log level to 1 to see what it did
idx = ImageD11.indexing.index( ui.colfile, npk_tol=[( ui.colfile.nrows//2, 0.05),], log_level=3)
print('UBIs found:')
idx.ubis

In [ ]:
idx.saveubis(ubi_file)

# Refining indexed crystal UBI and translation with initial parameters

In [ ]:
!makemap.py -f {flt_file} -u {ubi_file} -U {map_file} -p {par_file_powder} -l {symmetry} -s {symmetry} -t 0.05 --omega_slop={dset.ostep/2}
!makemap.py -f {flt_file} -u {map_file} -U {map_file} -p {par_file_powder} -l {symmetry} -s {symmetry} -t 0.025 --omega_slop={dset.ostep/2}

# Fitting parameters with refined crystal UBI and translation

In [ ]:
# fix the wedge to zero
!refine_em.py {new_flt_file} {map_file} {par_file_powder} --omega_slop={dset.ostep/2} -x wedge -l {symmetry}

# Refining indexed crystal UBI and translation with refined parameters

In [ ]:
# refine_em.py creates 0.par

!cp 0.par {par_file_end}
!makemap.py -f {flt_file} -u {map_file} -U {map_file} -p {par_file_end} -l {symmetry} -s {symmetry} -t 0.05 --omega_slop={dset.ostep/2}
!makemap.py -f {flt_file} -u {map_file} -U {map_file} -p {par_file_end} -l {symmetry} -s {symmetry} -t 0.025 --omega_slop={dset.ostep/2}

# Re-fitting parameters with refined crystal UBI and translation

In [ ]:
!refine_em.py {new_flt_file} {map_file} {par_file_end} --omega_slop={dset.ostep/2} -x wedge -l {symmetry}

# Final refinement of indexed crystal UBI and translation

In [ ]:
!cp 0.par {par_file_end}
!makemap.py -f {flt_file} -u {map_file} -U {map_file} -p {par_file_end} -s {symmetry} -t 0.025 --omega_slop={dset.ostep/2}

# Look at new parameters

In [ ]:
!cat {par_file_end}

In [ ]:
!ubi2cellpars.py {map_file}

In [ ]:
g = ImageD11.grain.read_grain_file(map_file)[0]
v  = np.linalg.det(g.ubi)
a_avg = pow(v ,1/3)

deviatoric = g.eps_grain_matrix( [a_avg, a_avg, a_avg, 90, 90, 90] )
print('deviatoric strains, should be zero, so an estimate of precision:\n',deviatoric)

In [ ]:
wold = ui.parameterobj.get('wavelength')
wnew = wold*a_silicon/a_avg
print( 'Wavelength input',wold,'estimated from silicon',wnew)
print( 'Energy input',econst/wold,'estimated from silicon',econst/wnew)

__If `wnew` is significantly different from `wold`, we recommend that you update the wavelength in the initial parameter guess, then re-running the rest of the notebook.__

# Final parameter visualisation

In [ ]:
newcolf = ImageD11.columnfile.columnfile(new_flt_file)
newcolf.parameters.loadparameters(par_file_end)
newcolf.updateGeometry()
ucell = ImageD11.unitcell.unitcell.from_par_file(par_file_end)
ucell.makerings(newcolf.ds.max())

fig, ax = pl.subplots(constrained_layout=True, figsize=(10, 5))
ax.vlines(np.degrees(2*np.arcsin(np.array(ucell.ringds) * newcolf.parameters.get('wavelength')/2)), -180, 180, color='orange', zorder=0)
ax.scatter(newcolf.tth_per_grain, newcolf.eta_per_grain, s=2)
ax.set_xlim(2,22)
ax.set_xlabel('2-theta ($^{o}$)')
ax.set_ylabel('$\eta$ ($^{o}$)')
pl.show()

__Once you're happy with the parameters, run the below cell to create a parameters json file with a geometry.par in the current directory__

In [ ]:
# load in the final parameters
final_pars = parameters.from_file(par_file_end)
# set crystal translations to zero
final_pars.set('t_x', 0.0)
final_pars.set('t_y', 0.0)
final_pars.set('t_z', 0.0)
final_pars.set('phase_name', 'Si')
# make new AnalysisSchema from these
asc = AnalysisSchema.from_old_pars_object(final_pars)
# save to disk
asc.save('pars.json')